# Text Generation


**Information**

**Tech Concept Generator**: Utilize LLMs to identify potential application scenarios for the "Soft Robotic Walker" (SRW) considering various combinations of options (highlighted in red).



*** 
**Background information**

* Conceptualization of current study with the demonstrator area in livmats.


***
**Coding sources**



***
**Aim of the code template**

Utilize LLMs to identify potential application scenarios for the "Soft Robotic Walker" (SRW) considering various combinations of options (highlighted in red on the slide).

## Get API key(s)

In [1]:
import os
import sys

# Assuming 'src' is one level down (in the current directory or a subdirectory)
path_to_src = os.path.join('../..','src')  # Moves one level down to 'src' folder

# Add the path to sys.path
sys.path.append(path_to_src)

# Now you can import your API_key module
import API_key as key

# Tech Concept Generator

## set up table of all possible combinations:

In [2]:
import itertools
import pandas as pd

# Define lists for each variable
softness = [
    'completely made of soft materials and thereby deformable',
    'completely made of rigid materials and thereby not deformable',
    'partly made of soft rigid materials and thereby partly deformable'
]
electronic = [
    'autonomously + electronic free (self-charging)',
    'external controlled + electronic'
]
motivation = [
    'existing principles in nature + bioinspired (like stick insect)',
    'existing principles of conventional robots'
]
developedBy = [
    'at the University',
    'in the industry'
]

# Generate Cartesian product
product = list(itertools.product(softness, electronic, motivation, developedBy))

# Convert to a pandas DataFrame
df = pd.DataFrame(product, columns=['softness', 'electronic', 'motivation', 'developedBy'])

# Add an ID column based on row index
df['ID'] = df.index

# Move ID column to the front
df = df[['ID', 'softness', 'electronic', 'motivation', 'developedBy']]

# Display the DataFrame
print(df.shape)
print(df)

(24, 5)
    ID                                           softness  \
0    0  completely made of soft materials and thereby ...   
1    1  completely made of soft materials and thereby ...   
2    2  completely made of soft materials and thereby ...   
3    3  completely made of soft materials and thereby ...   
4    4  completely made of soft materials and thereby ...   
5    5  completely made of soft materials and thereby ...   
6    6  completely made of soft materials and thereby ...   
7    7  completely made of soft materials and thereby ...   
8    8  completely made of rigid materials and thereby...   
9    9  completely made of rigid materials and thereby...   
10  10  completely made of rigid materials and thereby...   
11  11  completely made of rigid materials and thereby...   
12  12  completely made of rigid materials and thereby...   
13  13  completely made of rigid materials and thereby...   
14  14  completely made of rigid materials and thereby...   
15  15  complete

## set up prompt templates:

In [3]:
system_template = """
<Context>
You are an helpful AI designed to generate innovative technological application scenarios for the "Soft Robotic Walker" (SRW). 
The SRW has unique characteristics that can be combined in various ways to identify potential applications. 
The characteristics of the SRW are described using the following factors:

1. **Softness**: {softness}
2. **Electronic**: {electronic}
3. **Motivation**: {motivation}
4. **Developed By**: {developedBy}

Your task is to suggest applications for the SRW that align with the provided combinations of characteristics. 
Think creatively and consider implications of these characteristics to propose application scenarios that leverage these features.
</Context>
"""


user_template = """
<Task>
Generate one detailed application scenario for the SRW based on the given characteristics. 
The scenario should be specific, feasible, and illustrate how the SRW could be utilized in real-world or innovative settings. 
Ensure that the application reflects the combination of factors specified.

Here is an example format for the scenario output:

**Application Scenario**: [Detailed explanation of how these combined characteristics create a use case for the SRW]\\;

Provide the output in the this format, with each enumeration separated by a semicolon (\\;).
Thr scenario should be realistic or conceptually sound and reflect thoughtful alignment with the characteristics provided.
</Task>
"""


# Example invocation values for characteristics
example_values = {
    "softness": "partly made of soft rigid materials and thereby partly deformable",
    "electronic": "external controlled + electronic",
    "motivation": "existing principles in nature + bioinspired (Stabschrecke)",
    "developedBy": "at the University"
}

# Generate the output by invoking the templates
system_template_filled = system_template.format(**example_values)
user_template_filled = user_template

print("system_template_invoked:")
print(system_template_filled)
print("\nuser_template_invoked:")
print(user_template_filled)


system_template_invoked:

<Context>
You are an helpful AI designed to generate innovative technological application scenarios for the "Soft Robotic Walker" (SRW). 
The SRW has unique characteristics that can be combined in various ways to identify potential applications. 
The characteristics of the SRW are described using the following factors:

1. **Softness**: partly made of soft rigid materials and thereby partly deformable
2. **Electronic**: external controlled + electronic
3. **Motivation**: existing principles in nature + bioinspired (Stabschrecke)
4. **Developed By**: at the University

Your task is to suggest applications for the SRW that align with the provided combinations of characteristics. 
Think creatively and consider implications of these characteristics to propose application scenarios that leverage these features.
</Context>


user_template_invoked:

<Task>
Generate one detailed application scenario for the SRW based on the given characteristics. 
The scenario should 

## call LLMs to generate ideas

In [4]:
import pandas as pd
import re
from huggingface_hub import InferenceClient

# Initialize the client (assuming the client and API key are already set up)
client = InferenceClient(headers={"X-use-cache": "false"}, token=key.hugging_api_key)

def get_application_scenario(system_template_invoked, user_template_invoked):
    chat_completion = client.chat_completion(
        messages=[
            {"role": "system", "content": system_template_invoked},
            {"role": "user", "content": user_template_invoked}
        ],
        model="meta-llama/Meta-Llama-3.1-70B-Instruct",
        temperature=0.6,
        stream=False,
        max_tokens=1400
    )
    return chat_completion

def process_application_scenario(message_content, row):
    # Use regex to find the application scenario
    match = re.search(r'\*\*Application Scenario\*\*: (.*?)(?=\\;|\Z)', message_content, re.DOTALL)
    
    if not match:
        raise ValueError("The provided message content does not contain a valid 'Application Scenario'.")

    # Extract the application scenario
    application_scenario = match.group(1).strip()

    # Convert to DataFrame format and include row information
    df_scenarios = pd.DataFrame({
        "application_scenario": [application_scenario],
        "ID": row['ID'],
        "softness": row['softness'],
        "electronic": row['electronic'],
        "motivation": row['motivation'],
        "developedBy": row['developedBy']
    })

    return df_scenarios

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Print the current processing ID for tracking
    print(f"Processing Row {index}: ID {row['ID']}")

    # Set up the system and user templates
    system_template_invoked = system_template.format(
        softness=row['softness'],
        electronic=row['electronic'],
        motivation=row['motivation'],
        developedBy=row['developedBy']
    )
    
    user_template_invoked = user_template  # The user template is static as it refers to the task format

    # Get the application scenario from the LLM
    chat_completion = get_application_scenario(system_template_invoked, user_template_invoked)

    # Process the returned content into a DataFrame
    scenario_df = process_application_scenario(chat_completion.choices[0].message.content, row)

    # Combine the result into a master DataFrame
    if index == 0:
        combined_df = scenario_df
    else:
        combined_df = pd.concat([combined_df, scenario_df], ignore_index=True)

Processing Row 0: ID 0
Processing Row 1: ID 1
Processing Row 2: ID 2
Processing Row 3: ID 3
Processing Row 4: ID 4
Processing Row 5: ID 5
Processing Row 6: ID 6
Processing Row 7: ID 7
Processing Row 8: ID 8
Processing Row 9: ID 9
Processing Row 10: ID 10
Processing Row 11: ID 11
Processing Row 12: ID 12
Processing Row 13: ID 13
Processing Row 14: ID 14
Processing Row 15: ID 15
Processing Row 16: ID 16
Processing Row 17: ID 17
Processing Row 18: ID 18
Processing Row 19: ID 19
Processing Row 20: ID 20
Processing Row 21: ID 21
Processing Row 22: ID 22
Processing Row 23: ID 23


In [5]:
# Display or use the combined DataFrame
print("Combined DataFrame with Application Scenarios:")
combined_df

Combined DataFrame with Application Scenarios:


,application_scenario,ID,softness,electronic,motivation,developedBy
0,Search and Rescue in Confined or Hard-to-Reach...,0,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles in nature + bioinspired (l...,at the University
1,Search and Rescue in Disaster Scenarios; Utili...,1,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles in nature + bioinspired (l...,in the industry
2,Search and Rescue in Confined or Disaster-Stri...,2,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles of conventional robots,at the University
3,Wearable Soft Robotic Walker for Rehabilitatio...,3,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles of conventional robots,in the industry
4,"""Rehabilitation and Prosthetic Training for Lo...",4,completely made of soft materials and thereby ...,external controlled + electronic,existing principles in nature + bioinspired (l...,at the University
5,Soft Robotic Walker (SRW) for Search and Rescu...,5,completely made of soft materials and thereby ...,external controlled + electronic,existing principles in nature + bioinspired (l...,in the industry
6,Wearable Soft Robotic Walker for Rehabilitatio...,6,completely made of soft materials and thereby ...,external controlled + electronic,existing principles of conventional robots,at the University
7,Exosuit for Lower Limb Rehabilitation in Clini...,7,completely made of soft materials and thereby ...,external controlled + electronic,existing principles of conventional robots,in the industry
8,Autonomous Environmental Monitoring in Delicat...,8,completely made of rigid materials and thereby...,autonomously + electronic free (self-charging),existing principles in nature + bioinspired (l...,at the University
9,Autonomous Environmental Monitoring in Harsh T...,9,completely made of rigid materials and thereby...,autonomously + electronic free (self-charging),existing principles in nature + bioinspired (l...,in the industry


In [6]:
combined_df.to_excel("applicationScenarios_SRW.xlsx", index=False)

## The “Stuff” Strategy

In [13]:
# Filter the DataFrame to get rows with specific conditions
filtered_df = combined_df[combined_df['softness'] == "completely made of soft materials and thereby deformable"]

# Combine the application_scenario texts
scenario_list = " ".join(filtered_df['application_scenario'].tolist())

# Display the filtered DataFrame and the combined scenario text (optional)
print("Filtered DataFrame:")
filtered_df

Filtered DataFrame:


,application_scenario,ID,softness,electronic,motivation,developedBy
0,Search and Rescue in Confined or Hard-to-Reach...,0,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles in nature + bioinspired (l...,at the University
1,Search and Rescue in Disaster Scenarios; Utili...,1,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles in nature + bioinspired (l...,in the industry
2,Search and Rescue in Confined or Disaster-Stri...,2,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles of conventional robots,at the University
3,Wearable Soft Robotic Walker for Rehabilitatio...,3,completely made of soft materials and thereby ...,autonomously + electronic free (self-charging),existing principles of conventional robots,in the industry
4,"""Rehabilitation and Prosthetic Training for Lo...",4,completely made of soft materials and thereby ...,external controlled + electronic,existing principles in nature + bioinspired (l...,at the University
5,Soft Robotic Walker (SRW) for Search and Rescu...,5,completely made of soft materials and thereby ...,external controlled + electronic,existing principles in nature + bioinspired (l...,in the industry
6,Wearable Soft Robotic Walker for Rehabilitatio...,6,completely made of soft materials and thereby ...,external controlled + electronic,existing principles of conventional robots,at the University
7,Exosuit for Lower Limb Rehabilitation in Clini...,7,completely made of soft materials and thereby ...,external controlled + electronic,existing principles of conventional robots,in the industry


In [14]:
print("\nCombined Application Scenarios:")
print(scenario_list)


Combined Application Scenarios:
Search and Rescue in Confined or Hard-to-Reach Spaces - Utilizing the Soft Robotic Walker (SRW) for Enhanced Exploration and Detection Capabilities Search and Rescue in Disaster Scenarios; Utilizing the SRW's softness, it can be deployed in rubble-filled areas or collapsed buildings to navigate through tight spaces and debris, reducing the risk of further collapse; The electronic and self-charging capabilities enable the SRW to operate autonomously for extended periods, allowing it to search for survivors without the need for external power sources or human intervention; Inspired by the stick insect's ability to adapt to different environments, the SRW's bioinspired design enables it to climb over obstacles and adjust to changing terrain, making it an ideal candidate for navigating disaster scenarios; Developed by industry experts, the SRW's robust design and advanced sensors allow it to detect vital signs, such as heartbeats or breathing patterns, and 

In [ ]:
# System and user template setup
system_template = """
<Context>
You are an helpful AI designed to summarize innovative technological application scenarios for the "Soft Robotic Walker" (SRW). 
The single scenarios have unique characteristics that have been combined in various ways to identify potential applications. 
</Context>
"""

user_template = f"""
<Task>
Your task is to provide a list of possible applications of the SRW described in the application scenarios.
Start with the most important possible application, which have been highlighted in multiple of the following scenario texts:

{scenario_list}
</Task>
"""

In [ ]:
import pandas as pd
from huggingface_hub import InferenceClient

# Initialize the client (assuming the client and API key are already set up)
client = InferenceClient(headers={"X-use-cache": "false"}, token=key.hugging_api_key)

# Invoke the chat completion
chat_completion = client.chat_completion(
    messages=[
        {"role": "system", "content": system_template},
        {"role": "user", "content": user_template}
    ],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0.6,
    stream=False,
    max_tokens=1400
)

# Print the response
print("\nSummary of Application Scenarios:")
print(chat_completion.choices[0].message.content)

Filtered DataFrame:
                                application_scenario  ID  \
0  Search and Rescue in Confined or Hard-to-Reach...   0   
1  Search and Rescue in Disaster Scenarios; Utili...   1   
2  Search and Rescue in Confined or Disaster-Stri...   2   
3  Wearable Soft Robotic Walker for Rehabilitatio...   3   
4  "Rehabilitation and Prosthetic Training for Lo...   4   
5  Soft Robotic Walker (SRW) for Search and Rescu...   5   
6  Wearable Soft Robotic Walker for Rehabilitatio...   6   
7  Exosuit for Lower Limb Rehabilitation in Clini...   7   

                                            softness  \
0  completely made of soft materials and thereby ...   
1  completely made of soft materials and thereby ...   
2  completely made of soft materials and thereby ...   
3  completely made of soft materials and thereby ...   
4  completely made of soft materials and thereby ...   
5  completely made of soft materials and thereby ...   
6  completely made of soft materials and thereb

In [ ]:
# Assuming chat_completion has been defined and contains the LLM response
summary_content = chat_completion.choices[0].message.content

# Define the file path
file_path = 'summary_of_application_scenarios.txt'

# Write the content to a file
with open(file_path, 'w') as file:
    file.write(summary_content)

# Print confirmation
print(f"Summary content saved to {file_path}")